In [ ]:
!pip install datasets transformers[torch] seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk("drive/MyDrive/ner_dataset_conll_format_augmented")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = "savasy/bert-base-turkish-ner-cased"#["dbmdz/convbert-base-turkish-cased","savasy/bert-base-turkish-ner-cased","dbmdz/convbert-base-turkish-mc4-cased","dbmdz/bert-base-turkish-cased","dbmdz/distilbert-base-turkish-cased","Buseak/penn_berturk_0203_v5","Buseak/model_from_berturk_Feb_5_TrainTestSplit","alierenak/berturk_cased_ner"]
saved_model_name = "ner_model_"+model_name.replace("/","_")

In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=1e-4,
per_device_train_batch_size=32,
per_device_eval_batch_size=32,
num_train_epochs=20,
weight_decay=0.01,
save_strategy="epoch",
save_total_limit=2,  # Limit the total amount of checkpoints
greater_is_better=True,
metric_for_best_model="eval_f1",
load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# training_args = TrainingArguments(
#     "test-ner",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,  # Lower learning rate
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=15,  # Increase number of epochs
#     weight_decay=0.01,
#     gradient_accumulation_steps=4,  # Simulate larger batch size
#     save_total_limit=2,  # Limit the total amount of checkpoints
#     load_best_model_at_end=True,  # Load the best model at the end of training
#     lr_scheduler_type='linear',
#     warmup_steps=500,  # Warmup steps for learning rate scheduler
#     save_strategy="epoch"
# )

In [ ]:
# learning_rate_list = [1e-4,5e-4,2e-5,5e-5,2e-6]
# weight_decay_list = [0.0,0.01,0.03,0.05,0.1,0.2,0.3,0.5]

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForTokenClassification.from_pretrained(model_name,num_labels=5,ignore_mismatched_sizes=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import datasets
metric = datasets.load_metric("seqeval",trust_remote_code=True)
label_list = dataset["train"].features["ner_tags"].feature.names


In [ ]:
import numpy as np
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
    }

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
def compute_metrics2(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]

    true_labels_flat = [label for sublist in true_labels for label in sublist]
    pred_labels_flat = [label for sublist in predictions for label in sublist]

    f1 = f1_score(true_labels_flat, pred_labels_flat, average='weighted')
    precision = precision_score(true_labels_flat, pred_labels_flat, average='weighted')
    recall = recall_score(true_labels_flat, pred_labels_flat, average='weighted')
    accuracy = accuracy_score(true_labels_flat, pred_labels_flat)

    return {
   "precision": precision,
   "recall": recall,
   "f1": f1,
  "accuracy": accuracy,
    }



In [ ]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics2
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.401105,0.753820,0.869452,0.807517,0.852031
2,No log,0.402765,0.770746,0.842689,0.805114,0.865918
3,No log,0.492809,0.788509,0.842037,0.814394,0.871678
4,No log,0.591186,0.788344,0.838773,0.812777,0.866854
5,No log,0.636666,0.801869,0.840078,0.820529,0.867526
6,No log,0.657227,0.775214,0.767624,0.771400,0.856124
7,No log,0.720900,0.809464,0.837467,0.823227,0.866152
8,0.150100,0.670224,0.799373,0.832245,0.815478,0.865714
9,0.150100,0.767910,0.793939,0.855091,0.823382,0.869047
10,0.150100,0.782209,0.787765,0.823760,0.805361,0.860129


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/se

TrainOutput(global_step=1260, training_loss=0.06584586936330038, metrics={'train_runtime': 513.9287, 'train_samples_per_second': 77.832, 'train_steps_per_second': 2.452, 'total_flos': 5001158283938688.0, 'train_loss': 0.06584586936330038, 'epoch': 20.0})

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ANAT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-PRESENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-ABSENT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OBS-UNCERTAIN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.8664246797561646,
 'eval_precision': 0.7950169875424689,
 'eval_recall': 0.8534954407294832,
 'eval_f1': 0.8232189973614775,
 'eval_accuracy': 0.8808290155440415,
 'eval_runtime': 2.0492,
 'eval_samples_per_second': 121.997,
 'eval_steps_per_second': 3.904,
 'epoch': 20.0}

In [ ]:
# model_loaded = AutoModelForTokenClassification.from_pretrained("test-ner/checkpoint-300",num_labels=6,ignore_mismatched_sizes=True)
# trainer_temp = Trainer(
#     model=model_loaded,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )
# trainer_temp.evaluate(tokenized_datasets["test"])

In [ ]:
saved_model_name += "_impression_yok"
model.save_pretrained(f"/content/drive/MyDrive/{saved_model_name}")
#tokenizer.save_pretrained("tokenizer_"+model_name)
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
import json
config = json.load(open(f"/content/drive/MyDrive/{saved_model_name}/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open(f"/content/drive/MyDrive/{saved_model_name}/config.json","w"))
#model_fine_tuned = AutoModelForTokenClassification.from_pretrained(f"{saved_model_name}",num_labels=6)
#from transformers import pipeline
#nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


#example = "Michael jordan is one of the best NBA players."

#ner_results = nlp(example)

#print(ner_results)

## sezinarseven/turkish-medical-field-detection-8

learning_rate=1e-4,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
**'eval_f1': 0.8313750396951414,**


learning_rate=1e-4,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.05,
**'eval_f1': 0.830739917434106**

args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=1e-4,
per_device_train_batch_size=32,
per_device_eval_batch_size=32,
num_train_epochs=15,
weight_decay=0.01,
save_strategy="epoch",
save_total_limit=2,  # Limit the total amount of checkpoints
greater_is_better=True,
metric_for_best_model="eval_f1",
load_best_model_at_end=True,
)

{'eval_loss': 0.6016746759414673,
 'eval_precision': 0.7973300970873787,
 'eval_recall': 0.8713527851458885,
 **'eval_f1': 0.8326996197718631,**
 'eval_accuracy': 0.8776832892998679,
 'eval_runtime': 1.3595,
 'eval_samples_per_second': 151.527,
 'eval_steps_per_second': 5.149,
 'epoch': 15.0}

## dbmdz/convbert-base-turkish-cased

learning_rate=1e-4,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
**'eval_f1': 0.8257817485641353,**

learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
**'eval_f1': 0.8036103330220977**

args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=1e-4,
per_device_train_batch_size=32,
per_device_eval_batch_size=32,
num_train_epochs=15,
weight_decay=0.01,
save_strategy="epoch",
save_total_limit=2,  # Limit the total amount of checkpoints
greater_is_better=True,
metric_for_best_model="eval_f1",
load_best_model_at_end=True,
)

{'eval_loss': 0.47170954942703247,
 'eval_precision': 0.8008526187576127,
 'eval_recall': 0.8720159151193634,
 **'eval_f1': 0.8349206349206348,**
 'eval_accuracy': 0.8951637432829768,
 'eval_runtime': 1.7084,
 'eval_samples_per_second': 120.582,
 'eval_steps_per_second': 4.097,
 'epoch': 15.0}

## busecarik/berturk-sunlp-ner-turkish

learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
**'eval_f1': 0.7935523868567886,**

learning_rate=1e-4,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
**'eval_f1': 0.8354591836734694,**

learning_rate=1e-4,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.1,
**'eval_f1': 0.7876864487464297,**

learning_rate=1e-4,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0,
**'eval_f1': 0.8217153870864118,**

args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=1e-4,
per_device_train_batch_size=32,
per_device_eval_batch_size=32,
num_train_epochs=15,
weight_decay=0.01,
save_strategy="epoch",
save_total_limit=2,  # Limit the total amount of checkpoints
greater_is_better=True,
metric_for_best_model="eval_f1",
load_best_model_at_end=True,
)

{'eval_loss': 0.5507810711860657,
 'eval_precision': 0.7990226023213195,
 'eval_recall': 0.8673740053050398,
 **'eval_f1': 0.8317965023847378,**
 'eval_accuracy': 0.8839061691027121,
 'eval_runtime': 1.3583,
 'eval_samples_per_second': 151.661,
 'eval_steps_per_second': 5.154,
 'epoch': 15.0}

## alierenak/berturk_cased_ner

learning_rate=1e-4,                           
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01,
**'eval_f1': 0.8229002831078955,**

## savasy/bert-base-turkish-ner-cased

args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=1e-4,
per_device_train_batch_size=32,
per_device_eval_batch_size=32,
num_train_epochs=15,
weight_decay=0.01,
save_strategy="epoch",
save_total_limit=2,  # Limit the total amount of checkpoints
greater_is_better=True,
metric_for_best_model="eval_f1",
load_best_model_at_end=True,
)

{'eval_loss': 0.5417913198471069,
 'eval_precision': 0.7947650208209399,
 'eval_recall': 0.8859416445623343,
 **'eval_f1': 0.8378802132329884,**
 'eval_accuracy': 0.8917841089594106,
 'eval_runtime': 2.289,
 'eval_samples_per_second': 89.997,
 'eval_steps_per_second': 3.058,
 'epoch': 15.0}